In [1]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input,decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Layer,Lambda
import tensorflow.keras.backend as K
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
import cv2
from sklearn.model_selection import train_test_split

In [2]:
def target_category_loss_output_shape(input_shape):
    return input_shape

def target_category_loss(x, category_index, nb_classes):
    return tf.multiply(x, K.one_hot([category_index], nb_classes))

def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)

def load_image(path):
    img_path = sys.argv[1]
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

def register_gradient():
    if "GuidedBackProp" not in ops._gradient_registry._registry:
        @ops.RegisterGradient("GuidedBackProp")
        def _GuidedBackProp(op, grad):
            dtype = op.inputs[0].dtype
            return grad * tf.cast(grad > 0., dtype) * \
                tf.cast(op.inputs[0] > 0., dtype)

def compile_saliency_function(model, activation_layer='block5_conv3'):
    input_img = model.input
    layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])
    layer_output = layer_dict[activation_layer].output
    max_output = K.max(layer_output, axis=3)
    saliency = K.gradients(K.sum(max_output), input_img)[0]
    return K.function([input_img, K.learning_phase()], [saliency])

def modify_backprop(model, name):
    g = tf.get_default_graph()
    with g.gradient_override_map({'Relu': name}):

        # get layers that have an activation
        layer_dict = [layer for layer in model.layers[1:]
                      if hasattr(layer, 'activation')]

        # replace relu activation
        for layer in layer_dict:
            if layer.activation == keras.activations.relu:
                layer.activation = tf.nn.relu

        # re-instanciate a new model
        new_model = VGG16(weights='imagenet')
    return new_model

def deprocess_image(x):
    '''
    Same normalization as in:
    https://github.com/fchollet/keras/blob/master/examples/conv_filter_visualization.py
    '''
    if np.ndim(x) > 3:
        x = np.squeeze(x)
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.common.image_dim_ordering() == 'th':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

def _compute_gradients(tensor, var_list):
    grads = tf.gradients(tensor, var_list)
    return [grad if grad is not None else tf.zeros_like(var) for var, grad in zip(var_list, grads)]

In [3]:
def grad_cam(input_model, image, category_index, layer_name):
    '''
    Parameters
    ----------
    input_model : model
        評価するKerasモデル
    image : tuple等
        入力画像(枚数, 縦, 横, チャンネル)
    category_index : int
        入力画像の分類クラス
    layer_name : str
        最後のconv層の後のactivation層のレイヤー名.
        最後のconv層でactivationを指定していればconv層のレイヤー名.
        batch_normalizationを使う際などのようなconv層でactivationを指定していない場合は、
        そのあとのactivation層のレイヤー名.

    Returns
    ----------
    cam : tuple
        Grad-Camの画像
    heatmap : tuple
        ヒートマップ画像
    '''

    # ----- 1. 入力画像の予測クラスを計算 -----

    # 入力のcategory_indexが予想クラス

    # ----- 2. 予測クラスのLossを計算 -----
    # 分類クラス数
    nb_classes = 2

    # 入力データxのcategory_indexで指定したインデックス以外を0にする処理の定義
    target_layer = lambda x: target_category_loss(x, category_index, nb_classes)
    x = input_model.output
    x = Lambda(target_layer, output_shape=target_category_loss_output_shape)(x)
    model = tf.keras.models.Model([input_model.inputs], [input_model.get_layer(layer_name).output, input_model.output])
    #model.summary()

    with tf.GradientTape() as tape:
 
        # 引数のinput_modelの出力層の後にtarget_layerレイヤーを追加
        # modelのpredictをすると予測クラス以外の値は0になる
        # 予測クラス以外の値は0なのでsumをとって予測クラスの値のみ抽出
        conv_output, preds = model(image)
        class_idx = np.argmax(preds[0])
        loss = preds[:, class_idx]
        # 引数のlayer_nameのレイヤー(最後のconv層)のoutputを取得する
        # conv_output = [l for l in model.layers if l.name == layer_name][0].output
        # var_list = [conv_output]

    # ----- 3. 予測クラスのLossから最後のconv層への逆伝搬(勾配)を計算 -----

    # 予想クラスの値から最後のconv層までの勾配を計算する関数を定義
    # 定義した関数の
    # 入力 : [判定したい画像.shape=(1, 224, 224, 3)]、
    # 出力 : [最後のconv層の出力値.shape=(1, 14, 14, 512), 予想クラスの値から最後のconv層までの勾配.shape=(1, 14, 14, 512)]
    output =conv_output[0]
    grads = tape.gradient(loss,conv_output)[0]

    #gradient_function = K.function([model.input], [conv_output, grads[0]])
    gate_f = tf.cast(output > 0,'float32')
    gate_r = tf.cast(grads > 0,'float32')

    # 定義した勾配計算用の関数で計算し、データの次元を整形
    # 整形後
    # output.shape=(14, 14, 512), grad_val.shape=(14, 14, 512)
    #output, grads_val = gradient_function([image])
    #output, grads_val = output[0, :], grads_val[0, :, :, :]
    guided_grads = gate_f * gate_r * grads

    # ----- 4. 最後のconv層のチャンネル毎に勾配を平均を計算して、各チャンネルの重要度(重み)とする -----

    # weights.shape=(512, )
    # cam.shape=(14, 14)
    # ※疑問点1：camの初期化はzerosでなくて良いのか?
    weights = np.mean(guided_grads, axis = (0, 1))
    cam = np.dot(output, weights)
    #cam = np.ones(output.shape[0 : 2], dtype = np.float32)
    #cam = np.zeros(output.shape[0 : 2], dtype = np.float32)    # 私の自作モデルではこちらを使用

    # ----- 5. 最後のconv層の順伝搬の出力にチャンネル毎の重みをかけて、足し合わせて、ReLUを通す -----

    # 最後のconv層の順伝搬の出力にチャンネル毎の重みをかけて、足し合わせ
    #for i, w in enumerate(weights):
    #    cam += w * output[:, :, i]

    # 入力画像のサイズにリサイズ(14, 14) → (224, 224)
    cam = cv2.resize(cam, (210, 210))
    # 負の値を0に置換。処理としてはReLUと同じ。
    cam = np.maximum(cam, 0)
    # 値を0~1に正規化。
    # ※疑問2 : (cam - np.min(cam))/(np.max(cam) - np.min(cam))でなくて良いのか?
    heatmap = cam / np.max(cam)
    #heatmap = (cam - np.min(cam))/(np.max(cam) - np.min(cam))    # 私の自作モデルではこちらを使用

    # ----- 6. 入力画像とheatmapをかける -----

    # 入力画像imageの値を0~255に正規化. image.shape=(1, 224, 224, 3) → (224, 224, 3)
    #Return to BGR [0..255] from the preprocessed image
    image = image[0, :]
    image -= np.min(image)
    # ※疑問3 : np.uint8(image / np.max(image))でなくても良いのか?
    image = np.minimum(image, 255)

    # heatmapの値を0~255にしてカラーマップ化(3チャンネル化)
    cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)
    # 入力画像とheatmapの足し合わせ
    cam = np.float32(cam) + np.float32(image)
    # 値を0~255に正規化
    cam = 255 * cam / np.max(cam)
    return np.uint8(cam), heatmap

In [4]:
base_model = InceptionV3(include_top=False, weights='imagenet',input_shape=(210,210,3))
#plot_model(base_model,show_shapes=True,to_file='graph.png')

In [5]:
inp = tf.keras.layers.Input(shape=(210,210,3))

x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024,activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(2,activation='softmax')(x)

model = tf.keras.Model(inputs=base_model.input,outputs=x)
#plot_model(model,show_shapes=True,to_file='model.png')

In [6]:
for layer in model.layers[:249]:
    if layer.name.startswith('batch_normalization'):
        layer.trainable = True
    else:
        layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])

In [7]:
x0 = np.load('washer_OK.npy')
x1 = np.load('washer_NG_ponch.npy')
x2 = np.load('washer_NG_Scratch.npy')
X = np.concatenate([x0[:172],x1,x2],0)
X = X.reshape(344,210,210,1)
X = np.tile(X,(1,1,1,3))
y = np.concatenate([np.zeros((172)),np.ones((172))],0)
y = to_categorical(y)

# print(X.shape)

In [8]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.15)

In [9]:
history = model.fit(X_train,y_train,batch_size=96,epochs=30)

Epoch 1/30
4/4 [==============================] - 2s 411ms/step - loss: 2.9276 - accuracy: 0.5068
Epoch 2/30
4/4 [==============================] - 1s 200ms/step - loss: 1.0226 - accuracy: 0.6918
Epoch 3/30
4/4 [==============================] - 1s 201ms/step - loss: 0.4073 - accuracy: 0.7911
Epoch 4/30
4/4 [==============================] - 1s 200ms/step - loss: 0.2641 - accuracy: 0.8699
Epoch 5/30
4/4 [==============================] - 1s 200ms/step - loss: 0.1015 - accuracy: 0.9692
Epoch 6/30
4/4 [==============================] - 1s 200ms/step - loss: 0.1460 - accuracy: 0.9178
Epoch 7/30
4/4 [==============================] - 1s 201ms/step - loss: 0.0850 - accuracy: 0.9863
Epoch 8/30
4/4 [==============================] - 1s 200ms/step - loss: 0.2048 - accuracy: 0.9452
Epoch 9/30
4/4 [==============================] - 1s 202ms/step - loss: 0.2242 - accuracy: 0.9110
Epoch 10/30
4/4 [==============================] - 1s 202ms/step - loss: 0.1494 - accuracy: 0.9692
Epoch 11/30
4/4 [==

In [10]:
import pprint
names = [l.name for l in model.layers]
pprint.pprint(names,compact=True)

['input_1', 'conv2d', 'batch_normalization', 'activation', 'conv2d_1',
 'batch_normalization_1', 'activation_1', 'conv2d_2', 'batch_normalization_2',
 'activation_2', 'max_pooling2d', 'conv2d_3', 'batch_normalization_3',
 'activation_3', 'conv2d_4', 'batch_normalization_4', 'activation_4',
 'max_pooling2d_1', 'conv2d_8', 'batch_normalization_8', 'activation_8',
 'conv2d_6', 'conv2d_9', 'batch_normalization_6', 'batch_normalization_9',
 'activation_6', 'activation_9', 'average_pooling2d', 'conv2d_5', 'conv2d_7',
 'conv2d_10', 'conv2d_11', 'batch_normalization_5', 'batch_normalization_7',
 'batch_normalization_10', 'batch_normalization_11', 'activation_5',
 'activation_7', 'activation_10', 'activation_11', 'mixed0', 'conv2d_15',
 'batch_normalization_15', 'activation_15', 'conv2d_13', 'conv2d_16',
 'batch_normalization_13', 'batch_normalization_16', 'activation_13',
 'activation_16', 'average_pooling2d_1', 'conv2d_12', 'conv2d_14', 'conv2d_17',
 'conv2d_18', 'batch_normalization_12', 'ba

In [11]:
for i in range(X_test.shape[0]):
    pre_inp = X_test[i,:,:,:].reshape(-1,210,210,3)
    pred = model.predict(pre_inp)
    pred_Class = np.argmax(pred)
    cam,hm = grad_cam(model,pre_inp,pred_Class,'mixed10')
    cv2.imwrite('hm_'+ str(i) +'.png', cam)